In [1]:
import os
import matplotlib.pyplot as plt
from dataset import dataset
from edsr import edsr
from train import EdsrTrainer
from numba import cuda
from common import resolve
from common import resolve_single
from common import evaluate

In [2]:
# set hp depth and scale and get device to clear memory later

device = cuda.get_current_device()
depth = 16
scale = 4

In [3]:
# set directory for saving model weights 

weights_dir = './weights/edsr-16-x4/'
weights_file = os.path.join(weights_dir, 'weightsM16.h5')
os.makedirs(weights_dir, exist_ok=True)

In [4]:
# create our training and validation datasets

train_ds = dataset(batch_size=16, random_transform=True, subset='train')
valid_ds = dataset(batch_size=1, repeat_count=1, random_transform=False, subset='valid')

In [5]:
# initialize model and get weights from model trained by Krasserm

model=edsr(scale=scale, num_res_blocks=depth)
model.load_weights('weights/edsr-16-x4/weights.h5')     

In [1]:
# initialize the trainer

trainer = EdsrTrainer(model=model, checkpoint_dir=f'.ckpt/edsr-{depth}-x{scale}')

NameError: name 'EdsrTrainer' is not defined

In [ ]:
# train!

trainer.train(train_ds, valid_ds.take(10), steps=80000, evaluate_every=1000, save_best_only=True)

1000/80000: loss = 7.760, PSNR = 24.525671 (111.96s)
2000/80000: loss = 7.105, PSNR = 25.011930 (109.98s)
3000/80000: loss = 6.797, PSNR = 25.332977 (110.68s)
4000/80000: loss = 6.575, PSNR = 25.601933 (110.73s)
5000/80000: loss = 6.417, PSNR = 25.780127 (110.87s)
6000/80000: loss = 6.277, PSNR = 25.977528 (110.82s)


In [8]:
# restore the weights from the best checkpoint

trainer.restore()

Model restored from checkpoint at step 41000.


In [9]:
# validate results

psnrv = trainer.evaluate(valid_ds)
print(f'PSNR = {psnrv.numpy():3f}')

PSNR = 25.180470


In [10]:
# save weights

trainer.model.save_weights(weights_file)

In [1]:
# clear GPU memory

device.reset()

NameError: name 'device' is not defined